In [2]:
import pandas as pd 
import numpy as np

import seaborn as sn
import confusion_matrix_pretty_print
from confusion_matrix_pretty_print import plot_confusion_matrix_from_data
from sklearn import tree
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import confusion_matrix,classification_report,precision_score


import keras
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Softmax, Dropout
from keras import optimizers
from keras import metrics as kmetr
from keras.utils import plot_model

In [3]:
tripleData = pd.read_csv('../../triple42702_ShuffledPaired.csv')
zerodatad = pd.read_csv('../../probZeroDrug30000.csv')

In [4]:
fold = 10 
interval = int(42702/fold)
zeroInterval = int(30000/fold)

auprListZero = []
aucListZero = []

aucListpositive1 = []
auprListpositive1 = []

aucListmines1 = []
auprListmines1 = []

In [ ]:
#from sklearn.metrics import precision_recall_curve, roc_curve
from sklearn.metrics import auc, precision_recall_curve, roc_curve
from keras.utils import to_categorical


for split1 in range(0,10):
    split2 = split1 + 1
    
    trainIndex = list(range(0,split1 * interval))
    trainIndex.extend(list(range(split2 * interval,42702)))
    zeroTrainIndex = list(range(0,split1 * zeroInterval))
    zeroTrainIndex.extend(list(range(split2 * zeroInterval,30000)))

    if split1==9:
        testIndex = list(range(split1 * interval,42702))
        zeroTestIndex = list(range(split1 * zeroInterval,30000))

    else: 
        testIndex = list(range(split1 * interval,split2 * interval))
        ZeroTestIndex = list(range(split1 * zeroInterval,split2 * zeroInterval))
    
    dataTrain = tripleData.iloc[trainIndex,:].append(zerodatad.iloc[zeroTrainIndex,:], ignore_index = True)
    dataTest = tripleData.iloc[testIndex,:].append(zerodatad.iloc[ZeroTestIndex,:], ignore_index = True)

#     print(dataTrain.shape)
    print(min(testIndex),max(testIndex))
#     print(len(dataTest.iloc[:,0]), len(dataTrain.iloc[:,0]))


    X_train, X_test = dataTrain.values[:,3:], dataTest.values[:,3:]
    y_train, y_test = dataTrain.values[:,2].astype(int), dataTest.values[:,2].astype(int)
    trainNum = len(X_train)
    testNum = len(X_test)

    #reshape data to fit model
    X_train = X_train.reshape(trainNum,16,71,1)
    X_test = X_test.reshape(testNum,16,71,1)

    y_train = y_train + 1
    y_test  = y_test + 1

    # print(y_train[0], y_test[0])


    #one-hot encode target column
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    print(y_train[0],y_test[0])
    #create model
    #create model
    model = Sequential()
    #add model layers
    # kernel_initializer='uniform',
    # kernel_initializer='uniform',
    # kernel_initializer='uniform',
    # kernel_initializer='uniform',
    model.add(Conv2D(128, kernel_size=4, activation='relu', input_shape=(16,71,1)))
    # model.add(Conv2D(64, kernel_size=2, activation='relu'))

    model.add(Conv2D(32, kernel_size=4, activation='relu'))
    # model.add(Conv2D(16, kernel_size=2, activation='relu'))
    model.add(Conv2D(8, kernel_size=4, activation='relu'))
    model.add(Flatten())
    model.add(Dense( 64, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense( 16, activation='relu'))
    model.add(Dense( 3, activation='sigmoid'))
    # model.add(Softmax(128))
    model.summary()

    #compile model using accuracy to measure model performance


    adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999)
    # model.compile(loss='hinge', optimizer=adam, metrics=[kmetr.categorical_accuracy])
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


    # # Load the model's saved weights.
    # model.load_weights('cnn.h5')

    #train the model
    model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=9)


    # Saveing the Model
    model.save_weights('Weight/10-foldCV_LastTripleDDI/cnnTriple_'+str(split2)+'.h5')


    #predict first 4 images in the test set
    predit = model.predict(X_test)

    #actual results for first 4 images in test set
    print(predit[:4])



    # Degressive Result
    prec, rec, thr = precision_recall_curve(y_test[:,0], predit[:,0])
    aupr_val = auc(rec, prec)
    fpr, tpr, thr = roc_curve(y_test[:,0], predit[:,0])
    auc_val = auc(fpr, tpr)
    aucListmines1.append(auc_val)
    auprListmines1.append(aupr_val)
    print(aupr_val,auc_val)

    # Enhancive Result
    prec, rec, thr = precision_recall_curve(y_test[:,2], predit[:,2])
    aupr_val = auc(rec, prec)
    fpr, tpr, thr = roc_curve(y_test[:,2], predit[:,2])
    auc_val = auc(fpr, tpr)
    aucListpositive1.append(auc_val)
    auprListpositive1.append(aupr_val)
    print(aupr_val , auc_val )
    
    # Zero Result
    prec, rec, thr = precision_recall_curve(y_test[:,1], predit[:,1])
    aupr_val = auc(rec, prec)
    fpr, tpr, thr = roc_curve(y_test[:,1], predit[:,1])
    auc_val = auc(fpr, tpr)
    aucListZero.append(auc_val)
    auprListZero.append(aupr_val)
    print(aupr_val , auc_val )    

0 4269
[0. 0. 1.] [0. 0. 1.]




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 13, 68, 128)       2176      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 65, 32)        65568     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 62, 8)          4104      
_________________________________________________________________
flatten_1 (Flatten)          (None, 3472)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                222272    
_________________________________________________________________
dropout_1 (Dropout)          (None, 64

Train on 65432 samples, validate on 7270 samples
Epoch 1/9
65432/65432 [==============================] - 418s 6ms/step - loss: 0.6473 - acc: 0.7142 - val_loss: 0.4849 - val_acc: 0.7956
Epoch 2/9
65432/65432 [==============================] - 396s 6ms/step - loss: 0.4399 - acc: 0.8170 - val_loss: 0.4115 - val_acc: 0.8235
Epoch 3/9
65432/65432 [==============================] - 394s 6ms/step - loss: 0.3677 - acc: 0.8452 - val_loss: 0.3554 - val_acc: 0.8520
Epoch 4/9
65432/65432 [==============================] - 389s 6ms/step - loss: 0.3147 - acc: 0.8694 - val_loss: 0.3262 - val_acc: 0.8693
Epoch 5/9
65432/65432 [==============================] - 408s 6ms/step - loss: 0.2712 - acc: 0.8892 - val_loss: 0.2992 - val_acc: 0.8828
Epoch 6/9
65432/65432 [==============================] - 392s 6ms/step - loss: 0.2424 - acc: 0.9035 - val_loss: 0.2817 - val_acc: 0.8945
Epoch 7/9
65432/65432 [==============================] - 365s 6ms/step - loss: 0.2188 - acc: 0.9126 - val_loss: 0.2635 - val_acc:

Train on 65432 samples, validate on 7270 samples
Epoch 1/9
65432/65432 [==============================] - 358s 5ms/step - loss: 0.6296 - acc: 0.7205 - val_loss: 0.4489 - val_acc: 0.8066
Epoch 2/9
65432/65432 [==============================] - 352s 5ms/step - loss: 0.4104 - acc: 0.8276 - val_loss: 0.3633 - val_acc: 0.8418
Epoch 3/9
65432/65432 [==============================] - 351s 5ms/step - loss: 0.3338 - acc: 0.8594 - val_loss: 0.3058 - val_acc: 0.8724
Epoch 4/9
65432/65432 [==============================] - 352s 5ms/step - loss: 0.2811 - acc: 0.8849 - val_loss: 0.2744 - val_acc: 0.8838
Epoch 5/9
65432/65432 [==============================] - 349s 5ms/step - loss: 0.2416 - acc: 0.9025 - val_loss: 0.2421 - val_acc: 0.9032
Epoch 6/9
65432/65432 [==============================] - 348s 5ms/step - loss: 0.2097 - acc: 0.9153 - val_loss: 0.2192 - val_acc: 0.9131
Epoch 7/9
65432/65432 [==============================] - 349s 5ms/step - loss: 0.1850 - acc: 0.9265 - val_loss: 0.2154 - val_acc:

Train on 65432 samples, validate on 7270 samples
Epoch 1/9
65432/65432 [==============================] - 367s 6ms/step - loss: 0.6391 - acc: 0.7054 - val_loss: 0.4584 - val_acc: 0.8063
Epoch 2/9
65432/65432 [==============================] - 359s 5ms/step - loss: 0.4156 - acc: 0.8242 - val_loss: 0.3804 - val_acc: 0.8355
Epoch 3/9
46400/65432 [====================>.........] - ETA: 1:41 - loss: 0.3436 - acc: 0.8562

In [ ]:
from scipy.stats import sem, t
from scipy import mean
confidence = 0.95
# data = [1, 2, 3, 4, 5]

naucList = len(aucListpositive1)
maucList = mean(aucListpositive1)
std_erraucList = sem(aucListpositive1)
haucList = std_erraucList * t.ppf((1 + confidence) / 2, naucList - 1)


nauprList = len(auprListpositive1)
mauprList = mean(auprListpositive1)
std_errauprList = sem(auprListpositive1)
hauprList = std_errauprList * t.ppf((1 + confidence) / 2, nauprList  - 1)



print("positive1: "+ str(maucList) + '  -+' + str(haucList), str(mauprList) + '  -+' + str(hauprList))

In [ ]:
mypd = []
mypd = pd.DataFrame(mypd)
mypd['AUC'],mypd['AUPR'] = aucListpositive1, auprListpositive1
mypd.to_csv('LastTripleModelresult(+1).csv')

In [ ]:

confidence = 0.95
# data = [1, 2, 3, 4, 5]

naucList = len(aucListmines1)
maucList = mean(aucListmines1)
std_erraucList = sem(aucListmines1)
haucList = std_erraucList * t.ppf((1 + confidence) / 2, naucList - 1)


nauprList = len(auprListmines1)
mauprList = mean(auprListmines1)
std_errauprList = sem(auprListmines1)
hauprList = std_errauprList * t.ppf((1 + confidence) / 2, nauprList  - 1)



print("mines1: "+ str(maucList) + '  -+' + str(haucList), str(mauprList) + '  -+' + str(hauprList))

In [ ]:
mypd = []
mypd = pd.DataFrame(mypd)
mypd['AUC'],mypd['AUPR'] = aucListmines1, auprListmines1
mypd.to_csv('LastTripleModelresult(-1).csv')

In [ ]:

confidence = 0.95
# data = [1, 2, 3, 4, 5]

naucList = len(aucListZero)
maucList = mean(aucListZero)
std_erraucList = sem(aucListZero)
haucList = std_erraucList * t.ppf((1 + confidence) / 2, naucList - 1)


nauprList = len(auprListZero)
mauprList = mean(auprListZero)
std_errauprList = sem(auprListZero)
hauprList = std_errauprList * t.ppf((1 + confidence) / 2, nauprList  - 1)


print("Zeros: "+ str(maucList) + '  -+' + str(haucList), str(mauprList) + '  -+' + str(hauprList))

In [ ]:
mypd = []
mypd = pd.DataFrame(mypd)
mypd['AUC'],mypd['AUPR'] = aucListZero, auprListZero
mypd.to_csv('LastTripleModelresult(Zero).csv')